In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install scikit-learn==1.0.2
!pip install imbalanced-learn==0.8.0

# Loading the Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,precision_score,recall_score
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from joblib import Parallel, delayed, parallel_backend
from tensorflow.keras.backend import clear_session
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

# Load the Data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Major Project/creditcard.csv")
df.head(5)

# Scaling the Amount and Time

In [ ]:
df[['Scaled_Time', 'Scaled_Amount']] = StandardScaler().fit_transform(df[['Time', 'Amount']])
df = df.drop(['Time', 'Amount'], axis=1)

# Handling the Imbalance in the dataset using SMOTE

In [ ]:
X = df.drop(['Class'], axis=1)
y = df['Class']
smote = SMOTE(sampling_strategy=0.2, random_state=42)
X_res, y_res = smote.fit_resample(X, y)

In [ ]:
len(X_res)

# Splitting the Dataset into Train and Test Set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=.2, random_state=42)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

# Training the XGBoost Model

In [ ]:
xgb_model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbosity=1
)

# Train the model
xgb_model.fit(X_train, y_train)

# Predicting the Values and Evaluation

In [ ]:
y_predicted = xgb_model.predict(X_test)
print('Classification report:\n', classification_report(y_test, y_predicted))
cm = confusion_matrix(y_true=y_test, y_pred=y_predicted)
print('Confusion matrix:\n', cm)
print('Accuracy:', accuracy_score(y_test, y_predicted))
precision = precision_score(y_test, y_predicted, pos_label=1.0)
print('precision : ',precision)
recall = recall_score(y_test, y_predicted, pos_label=1.0)
print('recall rate : ',recall)

# Saving the Model and Metrics

In [ ]:
import os
import seaborn as sns
output_path = "/content/drive/MyDrive/Major Project/Model_Outputs"
os.makedirs(output_path, exist_ok=True)

# Saving the classification report and confusion matrix
report = classification_report(y_test, y_predicted)
cm = confusion_matrix(y_true=y_test, y_pred=y_predicted)
accuracy = accuracy_score(y_test, y_predicted)
precision = precision_score(y_test, y_predicted)
recall = recall_score(y_test, y_predicted)

# Save metrics
metrics_file = os.path.join(output_path, "xgboost_metrics.txt")
with open(metrics_file, "w") as f:
    f.write("Classification Report:\n")
    f.write(report)
    f.write("\nConfusion Matrix:\n")
    f.write(str(cm))
    f.write(f"\nAccuracy: {accuracy:.4f}")
    f.write(f"\nPrecision: {precision:.4f}")
    f.write(f"\nRecall: {recall:.4f}")
print(f"Metrics saved at: {metrics_file}")

# Save confusion matrix plot
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
conf_matrix_plot_path = os.path.join(output_path, "xgboost_confusion_matrix.png")
plt.savefig(conf_matrix_plot_path)
plt.close()
print(f"Confusion matrix plot saved at: {conf_matrix_plot_path}")

# Save the XGBoost model
model_save_path = os.path.join(output_path, "xgboost_model.json")
xgb_model.save_model(model_save_path)
print(f"Model saved at: {model_save_path}")

# Loading the Model

In [ ]:
import joblib
import os
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,precision_score,recall_score, roc_auc_score,roc_curve,precision_recall_curve,auc,roc_curve,f1_score
import matplotlib.pyplot as plt
import seaborn as sns

# Load the saved XGBoost model
loaded_xgb_model = XGBClassifier()
model_save_path = "/content/drive/MyDrive/Major Project/Model_Outputs/xgboost_model.json"
loaded_xgb_model.load_model(model_save_path)

# Use the loaded model for predictions
y_loaded_predicted = loaded_xgb_model.predict(X_test)
y_loaded_probabilities = loaded_xgb_model.predict_proba(X_test)[:, 1]  # Probability for the positive class (1)

# Evaluate the loaded model
cr=classification_report(y_test, y_loaded_predicted)
print('Classification report:\n', cr)
cm_loaded = confusion_matrix(y_true=y_test, y_pred=y_loaded_predicted)
print('Confusion matrix:\n', cm_loaded)
accuracy=accuracy_score(y_test, y_loaded_predicted)
recall=recall_score(y_test, y_loaded_predicted)
precision=precision_score(y_test, y_loaded_predicted)
f1=f1_score(y_test, y_loaded_predicted)
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-Score: {f1:.3f}")

# ROC Curve and AUC-ROC
fpr, tpr, _ = roc_curve(y_test, y_loaded_probabilities)
auc_roc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC AUC = {auc_roc:.3f}", color="red")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="best")
plt.grid()
plt.show()

# Precision-Recall Curve and AUC-PRC
precision, recall, _ = precision_recall_curve(y_test, y_loaded_probabilities)
auc_prc = auc(recall, precision)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f"PRC AUC = {auc_prc:.3f}", color="blue")
plt.title("Precision-Recall Curve")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc="best")
plt.grid()
plt.show()

plt.figure(figsize=(8, 6))
sns.heatmap(cm_loaded, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
conf_matrix_path = os.path.join(output_path, 'confusion_matrix_iso.png')
plt.savefig(conf_matrix_path)
plt.close()